# Workforce planning

Job recommendation can be extended across all members of an organisation.

The skills of each individual is compared with the skills of each job profile and an optimal set of suggestions returned.

This notebook will demonstrate this functionality using the users and profiles created in the setup notebook. If you have not already gone through the setup notebook please do so before running the code below.

In [1]:
import json
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000/api/v1/"

In [2]:
# Load the credentials set in the setup notebook

with open("credentials.json") as fp:
    headers = json.load(fp)

In [3]:
# Get organization details

r = requests.get(base_url + "organizations/", headers=headers)
print("Status code", r.status_code)

print("Organisation response", r.json())

Status code 200
Organisation response {'organizations': [{'id': '011509e6-e80f-487d-a5b8-8242b1078055', 'name': 'Test organization'}]}


In [4]:
# Get organization ID for the test organisation created during setup
assert 'Test organization' in [i['name'] for i in r.json()["organizations"]], "Organisation not found, run setup notebook"
organization_id = [i for i in r.json()["organizations"] if i["name"] == "Test organization"][0]["id"]

In [5]:
# Get users

r = requests.get(base_url + f"organizations/{organization_id}/users/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())
all_users = r.json()

Status code 200
Response {'users': [{'email': 'Adam@test.local', 'id': '8bd5e6ae-70cd-4afc-9a60-47961ae7a819', 'name': 'Adam', 'profile': '190e60cb-958e-478e-90b5-1b57f908e16c'}, {'email': 'Beth@test.local', 'id': 'ad0ce93b-fad6-4c01-8e5f-2c63d13152cd', 'name': 'Beth', 'profile': '0c15a884-2706-450d-a4c7-42bf9cf53531'}, {'email': 'Chris@test.local', 'id': 'ca1e6a6c-77b1-4ae6-9a06-b214cdc253e0', 'name': 'Chris', 'profile': '621d72d9-d721-4aee-90d6-66abef9bb46f'}, {'email': 'Dani@test.local', 'id': '89c5d4e8-8899-40e5-86f2-fc1f5898eec1', 'name': 'Dani', 'profile': '827a2192-c14c-4d92-b910-93e9e7f07d6a'}]}


In [6]:
# List all profiles for an organisation

r = requests.get(base_url + f"organizations/{organization_id}/profiles/", headers=headers)
print("Status code", r.status_code)

#print("Response", r.json())

profile_map = {i["name"]:i["id"] for i in r.json()["profiles"]}
profile_map

Status code 200


{'Applications Engineer': 'b117cd55-be55-4a6b-a331-7c2a87ae3f5b',
 'Business Intelligence Analyst': '129ece23-acb1-43a9-8832-296e40dd0100',
 'Chief Information Security Officer': '1fc0ad92-978d-45d3-88b5-f27406d8ef3f',
 'Cloud architect': '0b119dda-6fe7-473d-9917-5d4e4aa35498',
 'Computer Hardware Engineers': 'd29f2ce9-966d-4a59-8c3b-896ca810824d',
 'Computer and Information Systems Managers': '7b99672e-0496-416b-acaf-d0a48d95db65',
 'Data Analyst': 'f1ce01f1-80af-4aa2-9452-61a62f7ba104',
 'Data Engineer': '95d1aa39-1e5a-4c94-9d7d-09c33ee1deac',
 'Data Scientist': '3763b638-327a-4d18-9b27-ed81c048d565',
 'Database Administrators': 'b5e4d422-ede5-40f8-9ee5-7859d0049fb2',
 'Database Architects': 'f9a71017-ab02-4585-9aa8-e45d86cfbe99',
 'DevOps Engineer': '7e6d39d5-95aa-467f-97e7-5d6c7dd78c4a',
 'Electrical Engineer': '0da909b1-beeb-41e0-b5a0-7daf1d72a12e',
 'Full stack developer': '38e24e07-9c89-4d2e-bfb3-6beda5c31a63',
 'Information Security Analysts': 'd3f13ef9-d327-49c5-9177-e7cc8ff16

## Getting recommendations

The workforce planning service expects 

* a list of users, or
* a list of groups to select user from
* a list of dictionaries containing target profile IDs, number needed, and a maximum distance


In [7]:
# Using the 4 users previously created find options for 3 roles each with 2 positions available

data = {"users": [i["id"] for i in all_users["users"]],
        "targets":[{"profile":profile_map["Applications Engineer"],
                            "number_needed":2,
                            "max_training":100},
                          {"profile":profile_map["Security Engineer"],
                            "number_needed":2,
                            "max_training":100},
                          {"profile":profile_map["Electrical Engineer"],
                            "number_needed":2,
                            "max_training":100}],
}
data

{'targets': [{'max_training': 100,
   'number_needed': 2,
   'profile': 'b117cd55-be55-4a6b-a331-7c2a87ae3f5b'},
  {'max_training': 100,
   'number_needed': 2,
   'profile': '3eb6205c-f7ce-40ea-899f-9e74ec64b49b'},
  {'max_training': 100,
   'number_needed': 2,
   'profile': '0da909b1-beeb-41e0-b5a0-7daf1d72a12e'}],
 'users': ['8bd5e6ae-70cd-4afc-9a60-47961ae7a819',
  'ad0ce93b-fad6-4c01-8e5f-2c63d13152cd',
  'ca1e6a6c-77b1-4ae6-9a06-b214cdc253e0',
  '89c5d4e8-8899-40e5-86f2-fc1f5898eec1']}

In [8]:
r = requests.post(base_url + f"organizations/{organization_id}/workforce_planning/", json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()

Status code 200


In [9]:
# Set up to display profile and user names rather than IDs

profile_inv_map = {v:k for k,v in profile_map.items()}
user_inv_map = {i['id']:i['name'] for i in all_users['users']}

In [10]:
# View results

print("Best match for each target")
for best_users in results['users_by_target']:
    print(profile_inv_map[best_users['profile']])
    print("=" * len(profile_inv_map[best_users['profile']]))
    for user in best_users['recommendations']:
        user_name = user_inv_map[user['user']]
        distance = user['distance']
        print(f"{user_name:<10}| {distance}")
    print()

Best match for each target
Applications Engineer
Dani      | 14.0
Adam      | 17.0
Beth      | 20.0
Chris     | 20.0

Security Engineer
Dani      | 0.0
Adam      | 17.0
Beth      | 20.0
Chris     | 20.0

Electrical Engineer
Beth      | 0.0
Adam      | 33.0
Chris     | 33.0
Dani      | 33.0



In [11]:
# View results

print("Best match for each user")
for best_targets in results['targets_by_user']:
    print(user_inv_map[best_targets['user']])
    print("=" * len(user_inv_map[best_targets['user']]))
    for target in best_targets['recommendations']:
        target_name = profile_inv_map[target['profile']]
        distance = target['distance']
        print(f"{target_name:<20}| {distance}")
    print()



Best match for each user
Adam
====
Applications Engineer| 17.0
Security Engineer   | 17.0
Electrical Engineer | 33.0

Beth
====
Electrical Engineer | 0.0
Applications Engineer| 20.0
Security Engineer   | 20.0

Chris
=====
Applications Engineer| 20.0
Security Engineer   | 20.0
Electrical Engineer | 33.0

Dani
====
Security Engineer   | 0.0
Applications Engineer| 14.0
Electrical Engineer | 33.0

